In [2]:
import numpy as np
from zipfile import ZipFile
import pickle
import re
import math
import torch
from torch import nn, optim
from scipy.spatial import distance
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load data

!wget -O x.zip "https://drive.google.com/uc?export=download&id=1221858Ognjuffo3Pv6QezSiBw8xBOBe-"
!wget -O equal.pth "https://drive.google.com/uc?export=download&id=1_RIKfYvv6AuBmpeHZv1Nb3e5_qgpxRh0"

with ZipFile('/content/x.zip', 'r') as f:
  f.extract('x.txt')

x_real = []
x_imag = []
x = []
with open('x.txt') as input_file:
    for line in input_file:
      temp = re.sub(r'[()]', "", line)
      temp = complex(temp)
      x.append((temp.real, temp.imag))
      x_real.append(temp.real)
      x_imag.append(temp.imag)

y_real = []
y_imag = []
#with open('drive/MyDrive/y.txt') as input_file:
with open('y.txt') as input_file:
    for line in input_file:
      temp = re.sub(r'[()]', "", line)
      temp = complex(temp)
      y_real.append(temp.real)
      y_imag.append(temp.imag)

--2023-02-01 08:42:17--  https://drive.google.com/uc?export=download&id=1221858Ognjuffo3Pv6QezSiBw8xBOBe-
Resolving drive.google.com (drive.google.com)... 74.125.69.100, 74.125.69.138, 74.125.69.139, ...
Connecting to drive.google.com (drive.google.com)|74.125.69.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-80-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nq8cd3uht0ue1c3q9sldg73t381070gt/1675240875000/00806258749903615801/*/1221858Ognjuffo3Pv6QezSiBw8xBOBe-?e=download&uuid=7d063410-e2a3-4225-b79e-fad93180d328 [following]
--2023-02-01 08:42:23--  https://doc-0s-80-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nq8cd3uht0ue1c3q9sldg73t381070gt/1675240875000/00806258749903615801/*/1221858Ognjuffo3Pv6QezSiBw8xBOBe-?e=download&uuid=7d063410-e2a3-4225-b79e-fad93180d328
Resolving doc-0s-80-docs.googleusercontent.com (doc-0s-80-docs.googleusercontent.com)... 74.125.70.132, 2607:f8b0:

In [4]:
# Reshaping

y_real_t = torch.Tensor(y_real).reshape(-1,1)
y_imag_t = torch.Tensor(y_imag).reshape(-1,1)
Y = torch.stack((y_real_t, y_imag_t), -1)

In [5]:
# Define nonlinear NN for equalizer

class NonlinEq(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(2, 50)
        self.relu1 = nn.ReLU()
        self.lin2 = nn.Linear(50, 20)
        self.relu2 = nn.ReLU()
        self.lin3 = nn.Linear(20,2)

    def forward(self, y):
        y = self.relu1(self.lin1(y))
        y = self.relu2(self.lin2(y))
        return self.lin3(y)[::2]

In [6]:
# Convert y_eq in two arrays so that it can be plot

nleq = torch.load('equal.pth')
y_eq = nleq(Y)
yeq = y_eq.detach().numpy()
yeq=yeq[:,0,:]
yeq_real = yeq[:,0]
yeq_imag = yeq[:,1]

In [8]:
# Calculate σ^2

sum = 0
for j in range(len(x)-1):
    a = (yeq_real[j], yeq_imag[j])
    b = (x_real[j], x_imag[j])
    dst = distance.euclidean(a, b)
    sum = sum + dst**2

sigma_s = sum/len(x)

In [9]:
# Create a list with the 16 different mapped symbols

map_sym = list(dict.fromkeys(x))

In [10]:
# Calculate posteriori probabilities (divide p file to small batches)

p_o = np.zeros((400000, len(map_sym)))

for j in range(400000):
    den = 0
    a = (yeq_real[j], yeq_imag[j])
    for k in range(len(map_sym)):
        c = map_sym[k]
        dist = distance.euclidean(a, c)
        den = den + math.exp(-(dist**2)/(2*sigma_s))
    for i in range(len(map_sym)):
        b = map_sym[i]
        dst = distance.euclidean(a, b)
        num = math.exp(-(dst**2)/(2*sigma_s))

        p_o[j][i] = num/den

In [11]:
p_tw = np.zeros((400000, len(map_sym)))

for j in range(400000,800000):
    den = 0
    a = (yeq_real[j], yeq_imag[j])
    for k in range(len(map_sym)):
        c = map_sym[k]
        dist = distance.euclidean(a, c)
        den = den + math.exp(-(dist**2)/(2*sigma_s))
    for i in range(len(map_sym)):
        b = map_sym[i]
        dst = distance.euclidean(a, b)
        num = math.exp(-(dst**2)/(2*sigma_s))

        p_tw[j-400000][i] = num/den

In [12]:
p_th = np.zeros((400000, len(map_sym)))

for j in range(800000,1200000):
    den = 0
    a = (yeq_real[j], yeq_imag[j])
    for k in range(len(map_sym)):
        c = map_sym[k]
        dist = distance.euclidean(a, c)
        den = den + math.exp(-(dist**2)/(2*sigma_s))
    for i in range(len(map_sym)):
        b = map_sym[i]
        dst = distance.euclidean(a, b)
        num = math.exp(-(dst**2)/(2*sigma_s))

        p_th[j-800000][i] = num/den

In [13]:
p_fo = np.zeros((400000, len(map_sym)))

for j in range(1200000,1600000):
    den = 0
    a = (yeq_real[j], yeq_imag[j])
    for k in range(len(map_sym)):
        c = map_sym[k]
        dist = distance.euclidean(a, c)
        den = den + math.exp(-(dist**2)/(2*sigma_s))
    for i in range(len(map_sym)):
        b = map_sym[i]
        dst = distance.euclidean(a, b)
        num = math.exp(-(dst**2)/(2*sigma_s))

        p_fo[j-1200000][i] = num/den

In [14]:
p_fi = np.zeros((400000, len(map_sym)))

for j in range(1600000,2000000):
    den = 0
    a = (yeq_real[j], yeq_imag[j])
    for k in range(len(map_sym)):
        c = map_sym[k]
        dist = distance.euclidean(a, c)
        den = den + math.exp(-(dist**2)/(2*sigma_s))
    for i in range(len(map_sym)):
        b = map_sym[i]
        dst = distance.euclidean(a, b)
        num = math.exp(-(dst**2)/(2*sigma_s))

        p_fi[j-1600000][i] = num/den

In [15]:
# Save result

p_o.tofile('p_o.dat')
p_tw.tofile('p_tw.dat')
p_th.tofile('p_th.dat')
p_fo.tofile('p_fo.dat')
p_fi.tofile('p_fi.dat')